In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial load 
littlerock_table_id = "opendatadbt.311.littlerock311"

littlerock311 = requests.get("https://data.littlerock.gov/resource/2x6n-j9fb.json?$limit=10000&$where=ticket_created_date_time>='2021-03-22T00:00:00.000'") 
littlerockjson = littlerock311.json()
littlerock_df_parsed = littlerock_df['geocoded_column'].apply(pd.Series)
littlerock_df_parsed = littlerock_df_parsed.drop(littlerock_df_parsed.columns[[0,1]], axis=1)
littlerock_df_cleaned = pd.concat([littlerock_df.drop(['geocoded_column'], axis=1), littlerock_df_parsed], axis=1)
littlerock_df_cleaned
cols = ['ticket_id','ticket_created_date_time','ticket_closed_date_time','issue_type','issue_sub_category','human_address','ticket_status','latitude','longitude']
littlerock_df_cleaned = pd.DataFrame(littlerock_df_cleaned, columns = cols)

In [59]:
#Incremental load
littlerock_table_id = "opendatadbt.311.littlerock311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(ticket_created_date_time), max(ticket_closed_date_time)
FROM `opendatadbt.311.littlerock311`
limit 10
"""

littlerockmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
littlerockmaxopendatedate = str(littlerockmaxopendatedataframe['f0_'][0])

littlerockmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
littlerockmaxclosedate = str(littlerockmaxclosedatedataframe['f1_'][0])

In [60]:
littlerockmaxclosedate

'2021-04-02T02:57:00.000'

In [61]:

littlerock_table_id = "opendatadbt.311.littlerock311"

littlerock311 = requests.get("https://data.littlerock.gov/resource/2x6n-j9fb.json?$limit=10000&$where=ticket_created_date_time>'"+littlerockmaxopendatedate+"' OR ticket_closed_date_time between '" +littlerockmaxclosedate+ "' AND '" + current_datetime + "'") 
littlerockjson = littlerock311.json()
littlerock_df = pd.DataFrame(littlerockjson)
littlerock_df_parsed = littlerock_df['geocoded_column'].apply(pd.Series)
littlerock_df_parsed = littlerock_df_parsed.drop(littlerock_df_parsed.columns[[0,1]], axis=1)
littlerock_df_cleaned = pd.concat([littlerock_df.drop(['geocoded_column'], axis=1), littlerock_df_parsed], axis=1)
littlerock_df_cleaned
cols = ['ticket_id','ticket_created_date_time','ticket_closed_date_time','issue_type','issue_sub_category','human_address','ticket_status','latitude','longitude']
littlerock_df_cleaned = pd.DataFrame(littlerock_df_cleaned, columns = cols)

In [62]:
littlerock_df_cleaned

,ticket_id,ticket_created_date_time,ticket_closed_date_time,issue_type,issue_sub_category,human_address,ticket_status,latitude,longitude
0,21-00004172,2021-01-14T13:24:00.000,2021-04-02T09:30:00.000,Code Violations,Rental Inspections,"{""address"": ""2108 S PARK ST"", ""city"": ""LITTLE ...",Closed,34.72924207,-92.29772872
1,21-00008610,2021-01-28T12:46:00.000,2021-04-02T09:31:00.000,Code Violations,Illegal Dumping,"{""address"": ""408 BOOKER ST"", ""city"": ""LITTLE R...",Closed,34.74779317,-92.30691378
2,21-00027512,2021-03-26T09:50:00.000,2021-04-02T15:41:00.000,Trash Complaints,Garbage Cart Issues,"{""address"": ""1917 ALLIS ST"", ""city"": ""LITTLE R...",Closed,34.73312127,-92.30920655
3,21-00028188,2021-03-29T10:25:00.000,2021-04-02T07:58:00.000,*** CURRENTLY UNMAPPED ***,Trash / Rubbish / Debris on Premise (at Mobile...,"{""address"": ""8318 DOYLE SPRINGS RD"", ""city"": ""...",Closed,34.67310693,-92.32957217
4,21-00028265,2021-03-29T11:17:00.000,2021-04-02T15:50:00.000,Trash Complaints,Garbage Cart Issues,"{""address"": ""1912 BROWN ST"", ""city"": ""LITTLE R...",Closed,34.73331678,-92.31075231
...,...,...,...,...,...,...,...,...,...
947,21-00018770,2021-03-04T13:36:00.000,2021-04-02T15:59:00.000,Trash Complaints,Garbage Cart Issues,"{""address"": ""8319 GEYER SPRINGS RD"", ""city"": ""...",Closed,34.67420789,-92.34377308
948,21-00026980,2021-03-25T09:34:00.000,2021-04-02T15:46:00.000,Trash Complaints,Garbage Cart Issues,"{""address"": ""3111 S SCHILLER ST"", ""city"": ""LIT...",Closed,34.71916887,-92.29692692
949,21-00026937,2021-03-25T08:32:00.000,2021-04-03T12:48:00.000,Trash Complaints,Knuckleboom Pick-Up,"{""address"": ""6 AMHERST CV"", ""city"": ""LITTLE RO...",Closed,34.75397001,-92.35062045
950,21-00029273,2021-03-30T13:10:00.000,2021-04-02T10:31:00.000,Trash Complaints,On-Call / Bulky Items / Special Pick-Up Request,"{""address"": ""1501 WASHINGTON ST"", ""city"": ""LIT...",Closed,34.73772496,-92.32382058


In [63]:
littlerock_table_id = "opendatadbt.311.littlerock311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('littlerock311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    littlerock_df_cleaned, table_ref, job_config=job_config
).result()  # Make an API request.


In [3]:
littlerock_table_id = "opendatadbt.311.littlerock311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "Little Rock"
FROM `opendatadbt.311.littlerock311`

"""

littlerockdatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

littlerockdatacount = littlerockdatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

littlerock_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    littlerockdatacount, table_ref, job_config=job_config
).result()  # Make an API request.